In [1]:
import pandas as pd
import numpy as np
from glob import glob
import datetime,time
import warnings
warnings.filterwarnings('ignore')

In [2]:
class Backtesting :
    # Entry Data Path
    def __init__(self, path):
        
        self.path = path
    
    # Future Buy Backtesting Data  when rsi cross 60 level uper side 
    def Buy(self):
        path = pd.DataFrame(glob(self.path) , columns = ['Location'])
        trade_log =pd.DataFrame(columns =['Entry_date_time' , 'Single' , 'Buy_price' , 'Exit_price' , 'Exit_date_time' ,'Hold','RSI_LTP','Info', 'Pnl'])
        for index,row in path.iterrows():
            print(index)
            data = pd.read_csv(row['Location'])
            buy_price = data[(data['RSI14'] >= 60)]['open'].iloc[0]
            buy_date_time = data[(data['RSI14'] >= 60)]['date'].iloc[0]
            year = data['Year'].iloc[0]
            exit_date_time = data['date'].iloc[-1]

            buy_data=data[data['date'] > buy_date_time].reset_index(drop = 'Frist')

            buy_data.drop(['low' , 'Date' , 'Time' , 'Year' , 'Month'],axis =1 , inplace = True)

            stop_loss = 40
            target = round(buy_price*1.02)
            exit_price = 0
            buy_exit_date_time = ''
            pnl = 0
            ltp=''
            rsi_ltp =''
            Signal = 'Buy'
            info =''

            for index,row_buy in buy_data.iterrows():
                #print(row_buy)
                rsi = row_buy['RSI14']

                # Stop loss hit 
                if (rsi < stop_loss) :
                    ltp=row_buy['close']
                    buy_exit_date_time = row_buy['date']
                    rsi_ltp = row_buy['RSI14']
                    pnl=ltp-buy_price
                    print('Stop Loss Hit You Can a new position on sell side')
                    info = ('Stop Loss Hit You Can a new position on sell side')
                    break

                # Hit the 2 % target

                elif(row_buy['high'] >=target):

                    ltp=row_buy['high']
                    buy_exit_date_time = row_buy['date']
                    rsi_ltp = row_buy['RSI14']
                    pnl=ltp-buy_price
                    print('Hit the 2% Profit Target')
                    info = ('Hit the 2% Profit Target')
                    break

                # Not Hit stop loss Position automate squre off last day of month 15:15 
                elif (row_buy['date'] == exit_date_time):
                    ltp=row_buy['close']
                    buy_exit_date_time = row_buy['date']
                    rsi_ltp = row_buy['RSI14']
                    pnl=ltp-buy_price
                    print('Not hit Stoploss position square off Month last trading 15:15')
                    info = ('Not hit Stoploss position square off Month last trading 15:15')
                    break

            trade_log = trade_log.append({'Entry_date_time' : buy_date_time , 'Single' : Signal , 'Buy_price': buy_price ,'Exit_price' : ltp ,'Exit_date_time' : buy_exit_date_time ,
                                           'Hold': (pd.to_datetime(buy_exit_date_time).day - pd.to_datetime(buy_date_time).day),'RSI_LTP':rsi_ltp, 'Info' : info, 'Pnl': pnl} , ignore_index = True)
        
            trade_log.to_csv(f'{year}_Future_buy_result.csv',index=False)
        return trade_log


    # Future sell Backtesting Data  when rsi cross 40 level down side
    def sell(self):
        path = pd.DataFrame(glob(self.path) , columns = ['Location'])
        trade_log_sell =pd.DataFrame(columns =['Entry_date_time' , 'Single' , 'Sell_price' , 'Exit_price' , 'Exit_date_time' ,'Hold','RSI_LTP','Info', 'Pnl'])
        for index,row in path.iterrows():
            print(index)
            data = pd.read_csv(row['Location'])
            exit_date_time = data['date'].iloc[-1]
            year = data['Year'].iloc[0]
            sell_price = data[data['RSI14'] <= 40]['open'].iloc[0]
            sell_entry_datetime = data[data['RSI14'] <= 40]['date'].iloc[0]
            sell_data = data[data['date'] > sell_entry_datetime].reset_index(drop = 'Frist')
            sell_data.drop(['Date' , 'Time' , 'Year' , 'Month' , 'high'],axis =1 , inplace = True)


            stop_loss = 60
            target = round(sell_price*0.98)
            #exit_price = 0
            sell_exit_date_time = ''
            pnl = 0
            ltp=''
            rsi_ltp =''
            Signal = 'Sell'
            info = ''

            for index,row_sell in sell_data.iterrows():
                rsi = row_sell['RSI14']
                # Stop loss hit 
                if (rsi > stop_loss) :
                    ltp = row_sell['close']
                    sell_exit_date_time = row_sell['date']
                    rsi_ltp = row_sell['RSI14']
                    pnl=sell_price - ltp
                    print('Stop Loss Hit You Can a new position on Buy side')
                    info =('Stop Loss Hit You Can a new position on Buy side')
                    break
                # Hit the 2 % target
                elif(row_sell['low'] <= target):
                    ltp=row_sell['low']
                    sell_exit_date_time = row_sell['date']
                    rsi_ltp = row_sell['RSI14']
                    pnl=sell_price - ltp
                    print('Hit the 2% Profit Target')
                    info = ('Hit the 2% Profit Target')
                    break
                # Not Hit stop loss Position automate squre off last day of month 15:15 
                elif (row_sell['date'] == exit_date_time):
                    ltp=row_sell['close']
                    sell_exit_date_time = row_sell['date']
                    rsi_ltp = row_sell['RSI14']
                    pnl=sell_price - ltp
                    print('Not hit Stoploss position square off Month last trading 15:15')
                    info = ('Not hit Stoploss position square off Month last trading 15:15')
                    break

            trade_log_sell = trade_log_sell.append({'Entry_date_time' : sell_entry_datetime , 'Single' : Signal , 'Sell_price': sell_price ,'Exit_price' : ltp ,'Exit_date_time' : sell_exit_date_time ,
                                                   'Hold': (pd.to_datetime(sell_exit_date_time).day - pd.to_datetime(sell_entry_datetime).day),'RSI_LTP':rsi_ltp, 'Info': info , 'Pnl': pnl} , ignore_index = True)


            trade_log_sell.to_csv(f'{year}_Future_sell_result.csv',index=False)
        return trade_log_sell


In [3]:
path = r'C:\Bank Nifty Future Data-60 time Frame\2015/*'

In [4]:
Backtesting = Backtesting(path)

In [5]:
Backtesting.Buy()

0
Hit the 2% Profit Target
1
Not hit Stoploss position square off Month last trading 15:15
2
Stop Loss Hit You Can a new position on sell side
3
Stop Loss Hit You Can a new position on sell side
4
Hit the 2% Profit Target
5
Stop Loss Hit You Can a new position on sell side
6
Hit the 2% Profit Target
7
Stop Loss Hit You Can a new position on sell side
8
Stop Loss Hit You Can a new position on sell side
9
Hit the 2% Profit Target
10
Stop Loss Hit You Can a new position on sell side
11
Hit the 2% Profit Target


,Entry_date_time,Single,Buy_price,Exit_price,Exit_date_time,Hold,RSI_LTP,Info,Pnl
0,2015-10-01 09:15:00,Buy,7986.90,8180.45,2015-10-06 09:15:00,5,73.553422,Hit the 2% Profit Target,193.55
1,2015-11-20 10:15:00,Buy,7821.10,7939.50,2015-11-30 15:15:00,10,60.162689,Not hit Stoploss position square off Month las...,118.40
2,2015-12-01 09:15:00,Buy,7958.15,7889.85,2015-12-03 09:15:00,2,37.913292,Stop Loss Hit You Can a new position on sell side,-68.30
3,2015-01-23 09:15:00,Buy,8827.95,8799.10,2015-01-30 13:15:00,7,39.827666,Stop Loss Hit You Can a new position on sell side,-28.85
4,2015-02-12 14:15:00,Buy,8650.65,8869.95,2015-02-16 09:15:00,4,77.470704,Hit the 2% Profit Target,219.30
5,2015-03-02 09:15:00,Buy,8954.60,8862.75,2015-03-05 12:15:00,3,38.195657,Stop Loss Hit You Can a new position on sell side,-91.85
6,2015-04-01 13:15:00,Buy,8512.90,8692.50,2015-04-07 09:15:00,6,75.787230,Hit the 2% Profit Target,179.60
7,2015-05-04 14:15:00,Buy,8293.25,8167.95,2015-05-06 09:15:00,2,35.652160,Stop Loss Hit You Can a new position on sell side,-125.30
8,2015-06-01 09:15:00,Buy,8417.25,8305.05,2015-06-02 12:15:00,1,33.562197,Stop Loss Hit You Can a new position on sell side,-112.20
9,2015-07-01 09:15:00,Buy,8376.25,8561.10,2015-07-07 09:15:00,6,69.230642,Hit the 2% Profit Target,184.85


In [6]:
Backtesting.sell()

0
Hit the 2% Profit Target
1
Hit the 2% Profit Target
2
Hit the 2% Profit Target
3
Not hit Stoploss position square off Month last trading 15:15
4
Hit the 2% Profit Target
5
Hit the 2% Profit Target
6
Hit the 2% Profit Target
7
Hit the 2% Profit Target
8
Hit the 2% Profit Target
9
Stop Loss Hit You Can a new position on Buy side
10
Hit the 2% Profit Target
11
Hit the 2% Profit Target


,Entry_date_time,Single,Sell_price,Exit_price,Exit_date_time,Hold,RSI_LTP,Info,Pnl
0,2015-10-27 09:15:00,Sell,8232.35,8044.60,2015-10-30 14:15:00,3,22.956936,Hit the 2% Profit Target,187.75
1,2015-11-02 09:15:00,Sell,8055.65,7772.80,2015-11-09 09:15:00,7,23.681802,Hit the 2% Profit Target,282.85
2,2015-12-03 09:15:00,Sell,7901.15,7735.30,2015-12-08 09:15:00,5,30.912157,Hit the 2% Profit Target,165.85
3,2015-01-30 13:15:00,Sell,8834.85,8821.45,2015-01-30 15:15:00,0,44.118149,Not hit Stoploss position square off Month las...,13.40
4,2015-02-02 09:15:00,Sell,8802.50,8551.95,2015-02-09 09:15:00,7,27.035837,Hit the 2% Profit Target,250.55
5,2015-03-05 12:15:00,Sell,8906.10,8725.40,2015-03-10 09:15:00,5,30.108368,Hit the 2% Profit Target,180.70
6,2015-04-16 09:15:00,Sell,8757.05,8548.45,2015-04-20 09:15:00,4,25.942894,Hit the 2% Profit Target,208.60
7,2015-05-06 09:15:00,Sell,8316.60,8142.30,2015-05-06 10:15:00,0,34.477735,Hit the 2% Profit Target,174.30
8,2015-06-02 12:15:00,Sell,8351.95,8173.10,2015-06-03 09:15:00,1,21.341168,Hit the 2% Profit Target,178.85
9,2015-07-08 10:15:00,Sell,8450.65,8444.20,2015-07-13 12:15:00,5,63.761173,Stop Loss Hit You Can a new position on Buy side,6.45
